<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/GRIDMET_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Alison Soong


# Import libraries

In [1]:
import ee # Import Earth Engine API

In [2]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=7awf-KRZYr_TqDi51fgNU_Umim7NB-LeV6-8JG9t4Ec&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhppAMe-qPLhDlGWVjO63M0JjBeFBaFniJrpOX7tU2z2-UvUiJOSrE

Successfully saved authorization token.


In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [4]:
try:
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap seaborn matplotlib
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
        !conda install seaborn matplotlib -y
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt

# Wildfire Risk and populations

In [21]:
california_area = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))
landcover = ee.ImageCollection('USGS/NLCD').select('landcover').filterDate("2016-01-01", "2017-01-01").first().clip(california_area)

In [48]:
dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date('2020-01-01', '2021-01-15'))

maximumTemperature = dataset.select('tmmx').mean().clip(california_area)

temp_params = {
  'min': 290.0,
  'max': 314.0,
  'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']
}

tempCur = maximumTemperature.select('tmmx')
tempThree = tempCur.updateMask(tempCur.gt(300))
tempTwo = tempCur.updateMask(tempCur.gt(294).And(tempCur.lte(300))) 
tempOne = tempCur.updateMask(tempCur.gt(288).And(tempCur.lte(294))) 
tempZero = tempCur.updateMask(tempCur.lte(288)) 

# individual EVI scoring
tempThree = tempThree.where(tempCur.gt(300), 3)
tempTwo = tempTwo.where(tempCur.gt(294).And(tempCur.lte(300)), 2)
tempOne = tempOne.where(tempCur.gt(288).And(tempCur.lte(294)), 1)
tempZero = tempZero.where(tempCur.lte(288), 0)

# combined EVI
tempCur = tempCur.where(tempCur.gt(300), 3)
tempCur = tempCur.where(tempCur.gt(294).And(tempCur.lte(300)), 2)
tempCur = tempCur.where(tempCur.gt(288).And(tempCur.lte(294)), 1)
tempCur = tempCur.where(tempCur.lte(288).And(tempCur.gt(4)), 0)

# mask for areas that are not affected (water, certain marshland, etc)
tempCur = tempCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))



In [66]:
dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date('2020-01-01', '2021-01-15'))

minHumid = dataset.select('rmin').mean().clip(california_area)

humid_params = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']
}

# 0 to 15 is score 3, 15 to 30 is score 2, 30 to 60 is score 1, above 60 is score 0
humidCur = minHumid.select('rmin')
humidFour = humidCur.updateMask(humidCur.lte(15))
humidThree = humidCur.updateMask(humidCur.gt(15).And(humidCur.lte(30))) 
humidTwo = humidCur.updateMask(humidCur.gt(30).And(humidCur.lte(60))) 
humidOne = humidCur.updateMask(humidCur.gt(60).And(humidCur.lte(80))) 
humidZero = humidCur.updateMask(humidCur.gt(80)) 

# individual EVI scoring
humidFour = humidFour.where(humidCur.lte(15), 4)
humidThree = humidThree.where(humidCur.gt(15).And(humidCur.lte(30)), 3)
humidTwo = humidTwo.where(humidCur.gt(30).And(humidCur.lte(60)), 2)
humidOne = humidOne.where(humidCur.gt(60).And(humidCur.lte(80)), 1)
humidZero = humidZero.where(humidCur.gt(80), 0)

# combined EVI
humidCur = humidCur.where(humidCur.lte(15), 4)
humidCur = humidCur.where(humidCur.gt(15).And(humidCur.lte(30)), 3)
humidCur = humidCur.where(humidCur.gt(30).And(humidCur.lte(60)), 2)
humidCur = humidCur.where(humidCur.gt(60).And(humidCur.lte(80)), 1)
humidCur = humidCur.where(humidCur.gt(80), 0)

# mask for areas that are not affected (water, certain marshland, etc)
humidCur = humidCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))



In [193]:
dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date('2020-01-01', '2021-01-15'))

windVel = dataset.select('vs').mean().clip(california_area)

wind_params = {
  'min': 0.0,
  'max': 30.0,
  'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']
}

# frequent gusts, of 25 mph or greater. Thus 0-5 is score 0, 6-25 is score 1, and >25 is score 2 for miles per hour
# thus for meters/secondo -> >11 is score 3, 6-11 is score , 2-6 is score 1, and 0-2 is score 0

windCur = windVel.select('vs')
windThree = windCur.updateMask(windCur.gte(25))
windTwo = windCur.updateMask(windCur.gte(15).And(windCur.lt(25))) 
windOne = windCur.updateMask(windCur.gte(5).And(windCur.lt(15))) 
windZero = windCur.updateMask(windCur.gte(0).And(windCur.lt(5))) 

# individual EVI scoring
windThree = windThree.where(windCur.gte(25), 3)
windTwo = windTwo.where(windCur.gte(15).And(windCur.lt(25)), 2)
windOne = windOne.where(windCur.gte(5).And(windCur.lt(15)), 1)
windZero = windZero.where(windCur.gte(0).And(windCur.lt(5)), 0)

# combined EVI
windCur = windCur.where(windCur.lt(3), 0)
windCur = windCur.where(windCur.gte(3).And(windCur.lt(5)), 1)
windCur = windCur.where(windCur.gte(5).And(windCur.lt(7)), 2)
windCur = windCur.where(windCur.gte(7), 3)

# mask for areas that are not affected (water, certain marshland, etc)
windCur = windCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))



In [194]:
# Map it!

Map = geemap.Map()

Map.add_basemap("SATELLITE")

roi = ee.Geometry.Point(-118.6304, 37.3003) # (long, lat)
Map.centerObject(roi, 6)
Map.setCenter(-115.356, 38.686, 5)

# Temperature
# Map.addLayer(maximumTemperature, temp_params, 'Maximum Temperature', False)
# Map.addLayer(tempCur, {'min': 0, 'max': 3, 'palette': ['ece2f0','a6bddb','1c9099']}, 'masked max temp')

# Humidity
# Map.addLayer(minHumid, humid_params, 'humidity', False)
# Map.addLayer(humidCur, {'min': 0, 'max': 4, 'palette': ['white','ece2f0','a6bddb','1c9099']}, 'masked min humid')

# Wind
# Map.addLayer(windVel, wind_params, 'wind vel', False)
# Map.addLayer(windCur, {'min': 0, 'max': 3, 'palette': ['ece2f0','a6bddb','1c9099']}, 'masked wind vel')

Map.addLayerControl()
Map